In [6]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd

In [2]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
servicekey = 'GCyZSspoE7iPOuN+4UKO10fOhJyXmQhnhGwke3KcD8v2qaWYLQkoGLNn7vTwbctZkR/3uEECJAykpO4UJxUwTQ=='
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : servicekey, 
                               quote_plus('pageNo') : '1', 
                               quote_plus('numOfRows') : '10', 
                               quote_plus('startCreateDt') : '20200310', 
                               quote_plus('endCreateDt') : '20200315' })

In [3]:
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read()
print(response_body)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>3.2396602365</accDefRate><accExamCnt>268212</accExamCnt><accExamCompCnt>251940</accExamCompCnt><careCnt>7300</careCnt><clearCnt>834</clearCnt><createDt>2020-03-15 10:01:22.22</createDt><deathCnt>75</deathCnt><decideCnt>8162</decideCnt><examCnt>16272</examCnt><resutlNegCnt>243778</resutlNegCnt><seq>74</seq><stateDt>20200315</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-15 10:01:22.22</updateDt></item><item><accDefRate>3.3180003364</accDefRate><accExamCnt>261335</accExamCnt><accExamCompCnt>243701</accExamCompCnt><careCnt>7402</careCnt><clearCnt>714</clearCnt><createDt>2020-03-14 10:29:11.11</createDt><deathCnt>72</deathCnt><decideCnt>8086</decideCnt><examCnt>17634</examCnt><resutlNegCnt>235615</resutlNegCnt><seq>73</seq><stateDt>20200314</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-14 10:29:1

In [4]:
print(type(response_body))

print(request)

<class 'bytes'>


In [97]:
# ServiceKey : 공공데이터포털에서 발급받은 서비스키
# startCreateDt : 데이터 생성일 시작범위
# endCreateDt : 데이터 생성일 종료범위
params = {"ServiceKey" : parse.unquote(servicekey),
         'startCreateDt' : 20210920,
         "endCreateDt" : 20210927}

In [98]:
url = url + "?"
res = requests.get(url, params = params)
soup = bs(res.text, 'lxml')

In [99]:
print(soup)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item><accdefrate>2.2916883087</accdefrate><accexamcnt>14365935</accexamcnt><accexamcompcnt>13245824</accexamcompcnt><carecnt>31965</carecnt><clearcnt>269132</clearcnt><createdt>2021-09-27 10:08:41.366</createdt><deathcnt>2456</deathcnt><decidecnt>303553</decidecnt><examcnt>1120111</examcnt><resutlnegcnt>12942271</resutlnegcnt><seq>648</seq><statedt>20210927</statedt><statetime>00:00</statetime><updatedt>null</updatedt></item><item><accdefrate>2.2765913507</accdefrate><accexamcnt>14325938</accexamcnt><accexamcompcnt>13229076</accexamcompcnt><carecnt>30582</carecnt><clearcnt>268140</clearcnt><createdt>2021-09-26 09:38:04.296</createdt><deathcnt>2450</deathcnt><decidecnt>301172</decidecnt><examcnt>1096862</examcnt><resutlnegcnt>12927904</resutlnegcnt><seq>647</seq><statedt>20210926</statedt><statetime>00:00</statetime><updat

In [100]:
print(soup.find_all("decidecnt")[0].text)

303553


In [101]:
decidecnt_data = soup.find_all("decidecnt")
statedt_data = soup.find_all("statedt")

In [102]:
decidecnt_list = []
statedt_list = []
daycnt_list = []

In [103]:
for one in range(len(decidecnt_data)) :
    decidecnt_list.append(int(decidecnt_data[one].text))
    if one != 0 :
        statedt_list.append(statedt_data[one].text)

In [104]:
print("decidecnt len :", len(decidecnt_list))
print("statedt len :", len(statedt_list))

decidecnt len : 8
statedt len : 7


In [105]:
for i in range(len(decidecnt_data)) :
    print(decidecnt_list[i]) # 2381

for i in range(len(statedt_list)) :
    print(statedt_list[i])

for i in range(0, len(decidecnt_data) - 1) :
    print(decidecnt_list[i], decidecnt_list[i + 1])
    daycnt_list.append(decidecnt_list[i] - decidecnt_list[i + 1])

for i in daycnt_list :
    print(i)

303553
301172
298401
295132
292699
290983
289263
287534
20210926
20210925
20210924
20210923
20210922
20210921
20210920
303553 301172
301172 298401
298401 295132
295132 292699
292699 290983
290983 289263
289263 287534
2381
2771
3269
2433
1716
1720
1729


In [106]:
# 27 - 26 = 2381 -> 26일 데이터
corona_dic = {"날짜" : statedt_list, "일일 확진자수" : daycnt_list}
corona_df = pd.DataFrame(data = corona_dic)
corona_df

,날짜,일일 확진자수
0,20210926,2381
1,20210925,2771
2,20210924,3269
3,20210923,2433
4,20210922,1716
5,20210921,1720
6,20210920,1729


In [107]:
corona_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   날짜       7 non-null      object
 1   일일 확진자수  7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes
